In [1]:
().append(2)

AttributeError: 'tuple' object has no attribute 'append'

In [1]:
import json

with open('hb.json', 'r') as f:
    hb = json.load(f)
hb['CPTs']

[[{'visit': 0.01, 'no_visit': 0.99}],
 [{'VisitAsia': 'visit', 'present': 0.05, 'absent': 0.95},
  {'VisitAsia': 'no_visit', 'present': 0.01, 'absent': 0.99}],
 [{'yes': 0.5, 'no': 0.5}],
 [{'Smoking': 'yes', 'yes': 0.1, 'no': 0.9},
  {'Smoking': 'no', 'yes': 0.01, 'no': 0.99}],
 [{'Tubercolosis': 'present', 'Cancer': 'yes', 'yes': 1, 'no': 0},
  {'Tubercolosis': 'present', 'Cancer': 'no', 'yes': 1, 'no': 0},
  {'Tubercolosis': 'absent', 'Cancer': 'yes', 'yes': 1, 'no': 0},
  {'Tubercolosis': 'absent', 'Cancer': 'no', 'yes': 0, 'no': 1}],
 [{'TbOrCa': 'yes', 'abnormal': 0.98, 'normal': 0.02},
  {'TbOrCa': 'no', 'abnormal': 0.05, 'normal': 0.95}],
 [{'TbOrCa': 'yes', 'Bronchitis': 'yes', 'yes': 0.9, 'no': 0.1},
  {'TbOrCa': 'yes', 'Bronchitis': 'no', 'yes': 0.7, 'no': 0.3},
  {'TbOrCa': 'no', 'Bronchitis': 'yes', 'yes': 0.8, 'no': 0.2},
  {'TbOrCa': 'no', 'Bronchitis': 'no', 'yes': 0.1, 'no': 0.9}],
 [{'Smoking': 'yes', 'yes': 0.6, 'no': 0.4},
  {'Smoking': 'no', 'yes': 0.3, 'no': 0.7}]

In [2]:
import json

with open('hb.json', 'r') as f:
    hb = json.load(f)

In [3]:

import xarray as xr
import numpy as np

node_dict = {
    name:dict(
        name=name, states=states, parents=[], children=[], prob_table = [],
    ) for name, states in zip(hb['nodes'], hb['stateNames'])
}

for link in hb['linkList']:
    node_dict[link['Child']]['parents'].append(link['Parent'])
    node_dict[link['Parent']]['children'].append(link['Child'])

for i, cpt in enumerate(hb['CPTs']):
    node = list(node_dict.values())[i]
    coords = {n:dict(dims = [n], data=[state for state in node_dict[n]['states']]) for n in node['parents']}
    coords[node['name']] = dict(dims=[node['name']],data=['prob(' + state + ')' for state in node['states']])
    
    data_shape = [len(v['data']) for v in coords.values()]
    
    xr_dict = {
        "coords": coords,
        "dims": list(coords.keys()),
        "name": node['name'],
        "data": np.empty(data_shape)
    }

    xr_table = xr.DataArray.from_dict(xr_dict)
    
    for dct in cpt:
        dims = []
        found_number = False
        for key, value in dct.items():
            if isinstance(value, float) or isinstance(value, int):
                if not found_number:
                    dims.append('prob(' + key + ')')
                    found_number = True
                else:
                    dims[-1] = 'prob(' + key + ')'
                xr_table.loc[tuple(dims)] = value
            else:
                dims.append(value)
    node['prob_table'] = xr_table

            


In [4]:
node['prob_table']


('Smoking', 'Bronchitis')

In [17]:
xr_table

<xarray.DataArray 'TbOrCa' (Tubercolosis: 2, Cancer: 2, TbOrCa: 2)>
array([[[1.90000000e+001, 2.22809558e-312],
        [5.73116149e-322, 0.00000000e+000]],

       [[2.35541533e-312, 2.50395503e-312],
        [2.44029516e-312, 2.46151512e-312]]])
Coordinates:
  * Tubercolosis  (Tubercolosis) <U7 'present' 'absent'
  * Cancer        (Cancer) <U3 'yes' 'no'
  * TbOrCa        (TbOrCa) <U6 'p(yes)' 'p(no)'

In [7]:
a, (b, c) = [1, ['a', 'b']]

In [8]:
a = dict(a=1, b=2)

In [22]:
d = {
    "coords": {"tsafsda": {"dims": "t", "data": ['a' + str(x) for x in range(5)], "attrs": {"units": "s"}}},
    "attrs": {"title": "air temperature"},
    "dims": "t",
    "data": np.linspace(0,1,5),
    "name": "a",
}

import xarray as xr

xr.DataArray.from_dict(d)[0]

<xarray.DataArray 'a' ()>
array(0.)
Coordinates:
    tsafsda  <U2 'a0'
Attributes:
    title:    air temperature